In [ ]:
import wandb
import torch
import sklearn
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs

In [ ]:
# see GPU avaialability
cuda_available = torch.cuda.is_available()

In [ ]:
# import data
data = pd.read_csv('OneHot_Combined_cln_utf8.tsv', sep='\t')
data = data.iloc[-10000:,:]

# reformat data
sdg_lst = ['SDG1','SDG2','SDG3','SDG4','SDG5','SDG6','SDG7','SDG8','SDG9','SDG10','SDG11','SDG12','SDG13','SDG14','SDG15','SDG16','SDG17']
data['y'] = data[sdg_lst].values.tolist()
y = data['y']
X = data['abstract']

# plot ratio of data
class_weight = (data[sdg_lst].sum()/ data[sdg_lst].sum().sum())
print('% PER CLASS:\n\n', class_weight*100)
data[sdg_lst].sum().plot.bar()

# split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# add data to dataframes
train_df = pd.DataFrame()
train_df['text'] = X_train
train_df['labels'] = y_train
train_df.reset_index(inplace=True, drop=True)

eval_df = pd.DataFrame()
eval_df['text'] = X_val
eval_df['labels'] = y_val
eval_df.reset_index(inplace=True, drop=True)

# get number of classes
label_count = len(sdg_lst)

In [ ]:
# parameter optimisation
sweep_config = {
    "method": "bayes",  # bayes, grid, random
    "metric": {"name": "accuracy", "goal": "maximize"},
    "parameters": {
        "num_train_epochs": {"min": 1, "max": 20},
        "learning_rate": {"min": 5e-5, "max": 4e-4},
        "train_batch_size":{"min": 1, "max": 30},
        "eval_batch_size":{"min": 1, "max": 30},
        "warmup_steps":{"min": 50, "max": 500},
        "weight_decay":{"min": 0.01, "max": 0.1},
        # "logging_steps":{"min": 1, "max": 20}, #{"values": [2, 5, 10]}
        "threshold":{"min":0.0, "max":1.0}
    },
}

sweep_id = wandb.sweep(sweep_config, entity='sasdghub', project="sasdghub_ml_classify")

# logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
def acc_result(true, pred):
    true=np.argmax(true, axis=1)
    pred=np.argmax(pred, axis=1)
    acc = sklearn.metrics.accuracy_score(true, pred)
    return acc

def f1_macro_result(true, pred):
    true=np.argmax(true, axis=1)
    pred=np.argmax(pred, axis=1)
    f1 = sklearn.metrics.f1_score(true, pred, average='macro')
    return f1

def f1_micro_result(true, pred):
    true=np.argmax(true, axis=1)
    pred=np.argmax(pred, axis=1)
    f1 = sklearn.metrics.f1_score(true, pred, average='micro')
    return f1

def cm_result(true, pred):
    true=np.argmax(true, axis=1)
    pred=np.argmax(pred, axis=1)
    cm = wandb.plot.confusion_matrix(probs=None, y_true=true, preds=pred, class_names=sdg_lst) #sklearn.metrics.multilabel_confusion_matrix(true, pred)
    return cm

In [ ]:
# train with optimal parameters
# logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# # Optional model configuration
model_args = MultiLabelClassificationArgs(fp16= False,
                                          manual_seed = 4,
                                          use_multiprocessing = True,
                                          overwrite_output_dir=True,
                                          evaluate_during_training = True,
                                          num_train_epochs=20,
                                          learning_rate=0.0001,
                                          train_batch_size=20,
                                          eval_batch_size=20,
                                          warmup_steps=300,
                                          weight_decay=0.09,
                                          threshold=0.97,
                                          wandb_project = 'sasdghub_ml_classify',
                                          wandb_kwargs={
                                              'entity':'sasdghub'
                                                       }
                                         )

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
        "xlnet",
        "xlnet-base-cased",
        num_labels=label_count,
        args=model_args,
        use_cuda=cuda_available,
        pos_weight=list((1/label_count)/class_weight),
        # show_running_loss=True,
)

# Train the model
model.train_model(train_df,
                      verbose=True,
                      eval_df=eval_df,
                      accuracy=acc_result,
                      f1_macro=f1_macro_result,
                      f1_micro=f1_micro_result,
                      cm=cm_result)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df,
    verbose=True,
    accuracy=acc_result,
    f1_macro=f1_macro_result,
    f1_micro=f1_micro_result,
    cm=cm_result
)

In [ ]:
# Make predictions with the model
# predictions, raw_outputs = model.predict(["sanitation and clean drinking water"])

#######################################################################################


modify so it can digest longer text (split training data into paragraphs that can be digested) # needs an update

freeze number of layers as a parameter

select and save best model
apply new model in apply script

make testing script with ROC curves and confusion matrices (model.eval)

make testing dataset and train real classifier

get model embedding
train classifier for targets
OR
force nodes for each SDG to output only to targets (SDG7 node does not link to target 9.5 but only to 7.x targets.) (heirarichical classification NN)